In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'digit-recognizer:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F3004%2F861823%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240901%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240901T161842Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Dc8d818953692ac9aad196a3a879118835dc1f568aad5d507ce6aa88df3b4fb5f2934eda175963c9fa3f15662e1f28ae23d0e07365a3c33ca419598e7d535c2187fa8704817e5e31b734e2a1d1c636e6370d4fb6f745d025111f04669686f10ca6ed0775abc1d9306be653964982ac3f8b2f4d94f43178882d89433d7cc7bf34657819b5aaa0770ede8665a2b40b725cad30da438819b55592cf53e56d542a8eafb3e4aa2b2c8aae1e8ce52b24964084726434529a66fddbcca1e1cba7462e68a8265bca38e26c65898eda6d1080614973deef9d8c59bf3682ad32d79bad1628e63412d79846ac6499705e462927b4b368c17413dcf5f7fe861acd482202d7770'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn import svm
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Conv2D, MaxPool2D, Dropout
from keras import optimizers
import keras

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

2024-09-01 07:03:13.861935: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-01 07:03:13.862068: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-01 07:03:14.009289: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [ ]:
input1 = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
input2 = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")
print(input1.shape)
print(input2.shape)

(42000, 785)
(28000, 784)


In [ ]:
Y = input1['label'].to_numpy()
X = input1.iloc[:,1:].to_numpy()
X_submit = input2/255.0
Y_submit = pd.DataFrame(columns = ['ImageId', 'Label'])

In [ ]:
#CNN
X = X.reshape((-1, 28, 28, 1))
input2 = input2.to_numpy().reshape((-1, 28, 28, 1))
X.shape

(42000, 28, 28, 1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state=42)
X_train.shape

(33600, 28, 28, 1)

# 1. Multi class classification problem -> decision tree, svm, logistic + softmax, nn, ensemble

## Linear

In [ ]:
from sklearn.linear_model import LogisticRegression
Input1 = [ ('model', LogisticRegression() )]
#minmaxscaler has a bit increase accuracy than standardscaler
#/255.0 has a bit increase
pipe1 = Pipeline( Input1) #pipeline constructor
pipe1.fit( X_train/255.0, y_train)
y_predict = pipe1.predict(X_test/255.0)
print(accuracy_score(y_train, pipe1.predict(X_train/255.0)))
accuracy_score(y_test, y_predict)

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.942312925170068


0.918968253968254

## Non - linear

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
Input1 = [ ('scale', MinMaxScaler()),('model', KNeighborsClassifier(n_neighbors=3) )]
pipe1 = Pipeline( Input1) #pipeline constructor
pipe1.fit( X_train, y_train)
y_predict = pipe1.predict(X_test)
print(accuracy_score(y_train, pipe1.predict(X_train)))
accuracy_score(y_test, y_predict)

0.981938775510204


0.9663492063492064

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)
param_grid = [{'n_neighbors': [3, 5, 7, 9, 15, 20]}]
model = KNeighborsClassifier()
grid_search = GridSearchCV(model, param_grid, scoring="accuracy", n_jobs=-1, cv=kfold, verbose=1)
grid_result = grid_search.fit(X_train/255.0, y_train)

print(grid_result.best_score_, grid_result.best_params_)


Fitting 10 folds for each of 6 candidates, totalling 60 fits
0.9645918367346938 {'n_neighbors': 3}


In [ ]:
from sklearn.tree import DecisionTreeClassifier
Input1 = [ ('scale', MinMaxScaler()),('model', DecisionTreeClassifier(criterion = 'entropy', min_samples_split=4, min_samples_leaf=5) )]
    #overfitting -> cut tree earlier
pipe1 = Pipeline( Input1) #pipeline constructor
pipe1.fit( X_train, y_train)
y_predict = pipe1.predict(X_test)
print(accuracy_score(y_train, pipe1.predict(X_train)))
accuracy_score(y_test, y_predict)

0.9499404761904762


0.865

In [ ]:
param_grid = {
    'criterion': ['gini', 'entropy'],               # Criterion to measure the quality of a split
    'max_depth': [None, 10, 20, 30, 40],            # Maximum depth of the tree
    'min_samples_split': [2, 5, 10],                # Minimum number of samples required to split a node
    'min_samples_leaf': [1, 2, 4],                  # Minimum number of samples required to be at a leaf node
    'max_features': [None, 'sqrt', 'log2']          # Number of features to consider when looking for the best split
}
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

model = DecisionTreeClassifier()
grid_search = GridSearchCV(model, param_grid, scoring="accuracy", n_jobs=-1, cv=kfold, verbose=1)
grid_result = grid_search.fit(X_train/255.0, y_train)

print(grid_result.best_score_, grid_result.best_params_)


Fitting 5 folds for each of 270 candidates, totalling 1350 fits
0.8535374149659862 {'criterion': 'entropy', 'max_depth': None, 'max_features': None, 'min_samples_leaf': 4, 'min_samples_split': 5}


## SVM

In [ ]:
Input = [ ('scale', MinMaxScaler()),('model', svm.SVC(decision_function_shape='ovo'))]
pipe = Pipeline( Input) #pipeline constructor
pipe.fit( X_train, y_train)
y_predict = pipe.predict (X_test)
print(accuracy_score(y_train, pipe.predict (X_train)))
accuracy_score(y_test, y_predict)

0.9854761904761905


0.958452380952381

In [ ]:
model = svm.SVC(decision_function_shape='ovr')
model.fit(X_train/255.0, y_train)
y_predict = model.predict(X/255.0_test)
accuracy_score(y_test, y_predict)

0.973452380952381

In [ ]:
model = svm.SVC(decision_function_shape='ovo')
model.fit(X_train/255.0, y_train)
y_predict = model.predict(X_test/255.0)
print(accuracy_score(y_train, model.predict(X_train/255.0)))
accuracy_score(y_test, y_predict)

0.9886309523809523


0.973452380952381

In [ ]:
tmp = pd.DataFrame(data = pipe1.predict(input2))
Y_submit = tmp.reset_index().rename(columns = {'index':'ImageId',0: 'Label'})
Y_submit['ImageId'] += 1
Y_submit.head()

/opt/conda/lib/python3.10/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(


,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3


## Ensemble

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
Input1 = [ ('scale', MinMaxScaler()),('model', GradientBoostingClassifier(n_estimators = 100) )]
pipe1 = Pipeline( Input1) #pipeline constructor
pipe1.fit( X_train, y_train)
y_predict = pipe1.predict (X_test)
print(accuracy_score(y_train, pipe1.predict(X_train)))
accuracy_score(y_test, y_predict)
#so long

0.9697619047619047


0.9414285714285714

In [ ]:
from sklearn.ensemble import RandomForestClassifier
Input1 = [ ('scale', MinMaxScaler()),('model', RandomForestClassifier(n_estimators = 200, max_depth = 20, max_features = 4, min_samples_leaf = 5) )]
pipe1 = Pipeline( Input1) #pipeline constructor
pipe1.fit( X_train, y_train)
y_predict = pipe1.predict (X_test)
print(accuracy_score(y_train, pipe1.predict(X_train)))
accuracy_score(y_test, y_predict)

0.9746428571428571


0.9410714285714286

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
params = {
    'max_depth': [1, 2, 5, 10, 20],
    'min_samples_leaf': [5, 10, 20, 50, 100],
    'max_features': [2,3,4],
    'n_estimators': [10, 30, 50, 100, 200]
}

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

model = RandomForestClassifier()
grid_search = GridSearchCV(model, params, scoring="accuracy", n_jobs=-1, cv=kfold, verbose=1)
grid_result = grid_search.fit(X_train/255.0, y_train)

print(grid_result.best_score_, grid_result.best_params_)


Fitting 5 folds for each of 375 candidates, totalling 1875 fits
0.940267857142857 {'max_depth': 20, 'max_features': 4, 'min_samples_leaf': 5, 'n_estimators': 200}


# 2. With encoder label

In [ ]:
from sklearn.preprocessing import OneHotEncoder
one_hot = OneHotEncoder(sparse_output = False).fit(np.array([Y]).T)
Y_ = one_hot.transform(np.array([Y]).T)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y_, test_size = 0.2, random_state=42)
print(X_train.shape)
y_train.shape

(33600, 28, 28, 1)


(33600, 10)

## Fully NN

In [ ]:
X_train.shape[1]

784

In [ ]:
model = Sequential([Dense(512, input_dim=X_train.shape[1],  activation = 'relu', name='layer_1'),
                    Dense(128, activation='relu'), BatchNormalization(),
                    Dense(64, activation='relu'),
        Dense(10, activation='softmax'),]
)
model.compile(loss = 'categorical_crossentropy', optimizer = optimizers.Adam(), metrics = ['accuracy'])
model.fit( tf.convert_to_tensor(X_train/255.0),  tf.convert_to_tensor(y_train), epochs = 20)
model.evaluate(X_test/255.0, y_test, )

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.8801 - loss: 0.4063
Epoch 2/20
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.9638 - loss: 0.1166
Epoch 3/20
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.9755 - loss: 0.0795
Epoch 4/20
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.9815 - loss: 0.0593
Epoch 5/20
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.9839 - loss: 0.0476
Epoch 6/20
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.9882 - loss: 0.0361
Epoch 7/20
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.9887 - loss: 0.0348
Epoch 8/20
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.9888 - loss: 0.0308
Epoch 9/20
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.9925 - loss: 0.0240
Epoch 10/20
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.9919 - loss: 0.0237
Epoch 11/20
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.9935 - loss: 0.0188
Epoch 12/20
1050/1050 ━━━━━━━

[0.09918415546417236, 0.9788095355033875]

In [ ]:
X[0].shape

(28, 28, 1)

## CNN

In [ ]:
model = Sequential([keras.Input(shape=X_train[0].shape),
                    Conv2D(filters = 32, kernel_size=5, strides = 1, padding = 'same', activation = 'relu'),
                    BatchNormalization(),
                    MaxPool2D(),
                    BatchNormalization(),
                    Dropout(0.4),
                    Conv2D(filters = 64, kernel_size=5, strides = 1, padding = 'same', activation = 'relu'),
                    BatchNormalization(),
                    MaxPool2D(),
                    BatchNormalization(),
                    Dropout(0.4),
                     tf.keras.layers.Flatten(),
                    Dense(128, activation='relu'),
                    BatchNormalization(),
                    Dropout(0.4),
                    Dense(64, activation='relu'),
                    BatchNormalization(),
                    Dropout(0.4),
        Dense(10, activation='softmax')]
)
model.summary
model.compile(loss = 'categorical_crossentropy', optimizer = optimizers.Adam(), metrics = ['accuracy'])
model.fit( tf.convert_to_tensor(X_train/255.0),  tf.convert_to_tensor(y_train), epochs = 30)
model.evaluate(X_test/255.0, y_test, )

Epoch 1/30
  44/1050 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.2806 - loss: 2.3552 

W0000 00:00:1725174396.421098      84 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1050/1050 ━━━━━━━━━━━━━━━━━━━━ 14s 3ms/step - accuracy: 0.7660 - loss: 0.7419
Epoch 2/30
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9548 - loss: 0.1517
Epoch 3/30
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9639 - loss: 0.1200
Epoch 4/30
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9721 - loss: 0.0927
Epoch 5/30
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9734 - loss: 0.0873
Epoch 6/30
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9772 - loss: 0.0777
Epoch 7/30
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9780 - loss: 0.0746
Epoch 8/30
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9793 - loss: 0.0715
Epoch 9/30
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9804 - loss: 0.0694
Epoch 10/30
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9819 - loss: 0.0604
Epoch 11/30
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9836 - loss: 0.0567
Epoch 12/30
1050/1050 ━━━━━━━━━━━━━━━━━━

W0000 00:00:1725174498.718326      83 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


263/263 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9921 - loss: 0.0262


[0.021959824487566948, 0.9934523701667786]

# 3.

# II. Submission

In [ ]:
tmp = pd.DataFrame(data = one_hot.inverse_transform(model.predict(input2/255.0)).reshape([-1]))
print(tmp.head())
Y_submit = tmp.reset_index().rename(columns = {'index':'ImageId',0: 'Label'})
Y_submit['ImageId'] += 1
Y_submit.head()

101/875 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step

W0000 00:00:1725174509.064475      81 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


875/875 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step
   0
0  2
1  0
2  9
3  0
4  3


,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,0
4,5,3


In [ ]:
dir_submit = "/kaggle/working/submit1.csv"
Y_submit.to_csv(dir_submit, index = False)